In [ ]:
from google.colab import files
files.upload()


KeyboardInterrupt: 

In [ ]:
import os
os.listdir()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv("Bookings.csv")
df.head()


In [ ]:
df.info()
print()
df.columns

In [ ]:
df['Date'] = pd.to_datetime(df['Date'],errors='coerce')


In [ ]:
df.isnull().sum()


In [ ]:
#For cancelation columns
df['Canceled_Rides_by_Customer'] = df['Canceled_Rides_by_Customer'].fillna('NO')
df['Canceled_Rides_by_Driver'] = df['Canceled_Rides_by_Driver'].fillna('No')
#For ratings columns
ratings_df = df[df['Booking_Status'] == 'SUCCESS']
#For incomplete ride
df['Incomplete_Rides'] = df['Incomplete_Rides'].fillna('NO')
df['Incomplete_Rides_Reason'] = df['Incomplete_Rides_Reason'].fillna('NOT APPLICABLE')
#For incomplete payment
df['Payment_Method'] = df['Payment_Method'].fillna('NOT COMPLETED')
#For time based columns
tat_df = df[df['Booking_Status'] == 'SUCCESS']
#Drop unnecessary column
#df.drop(columns=['Unnamed: 20'],inplace=True)

#Check again
df.isnull().sum()


In [ ]:
#EDA
#1.Booking Trend Analysis
daily_bookings = df.groupby(df['Date'].dt.date).size()
daily_bookings.plot()
plt.title("Daily Ola Bookings Trend")
plt.xlabel("Date")
plt.ylabel("Bookings")
plt.show()

#2.Revenue Analysis by Vehicle Type
df.groupby('Vehicle_Type')['Booking_Value'].sum().plot(kind='bar')
plt.title("Revenue by Vehicle Type")
plt.show()

#3.Booking Status Analysis
df['Booking_Status'].value_counts().plot(kind='bar')
plt.title("Booking Status Distribution")
plt.show()

#4.Ratings Analysis
df[['Driver_Ratings', 'Customer_Rating']].describe()

#5.Payment Method Analysis
df['Payment_Method'].value_counts().plot(kind='bar')
plt.savefig("chart.png")
plt.show()


In [ ]:
#Cancellation Prediction Model (extra)
#STEP 1: Define the Target Variable
df['is_cancelled'] = df['Booking_Status'].apply(
    lambda x: 1 if x in ['Canceled by Driver', 'Canceled by Customer'] else 0
)
#STEP 2: Select Features (IMPORTANT)
features = df[['Vehicle_Type', 'Ride_Distance', 'Booking_Value', 'Payment_Method']]
target = df['is_cancelled']
#STEP 3: Handle Categorical Data (Encoding)
features_encoded = pd.get_dummies(features, drop_first=True)
#STEP 4: Train–Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    features_encoded, target, test_size=0.3, random_state=42
)
#STEP 5: Train the Model (Logistic Regression)
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
#STEP 6: Model Evaluation
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
df.to_csv("D:\\PowerBI\\Ola-Bookings-Analysis\\data\\ola_cleaned_data.csv", index=False)
from google.colab import files
files.download("ola_cleaned_data.csv")
